In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [13]:
from pathlib import Path
from sklearn.model_selection import train_test_split

In [15]:
# tokenizer

from transformers import BertTokenizer
from transformers import glue_convert_examples_to_features

In [16]:
# model

from transformers import BertForSequenceClassification
from transformers import TFBertForSequenceClassification

from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

from transformers import TFTrainer, TFTrainingArguments

## Data

### mrpc

In [ ]:
data = tfds.load('glue/mrpc', with_info=True)

In [ ]:
print(type(data))
len(data)

In [ ]:
print(type(data[0]))
print(type(data[1]))

In [ ]:
data[0]

In [ ]:
batch_size = 32
len(data[0]['train']) / batch_size

In [ ]:
len(data[0]['validation'])

In [ ]:
data[1]

### imdb

In [4]:
!wget -nc http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -P ./data

wget: /data2/wangyh/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
File ‘./data/aclImdb_v1.tar.gz’ already there; not retrieving.



In [24]:
# TODO: shell redirect > /dev/null

!tar -k -xf ./data/aclImdb_v1.tar.gz -C ./data

In [6]:
def read_imdb_split(split_dir):
    
    split_dir = Path(split_dir)
    texts = []
    labels = []
    
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
           
            # print(type(text_file))
            # print(text_file)
            # print(text_file.read_text())
            # raise ValueError(' ')
            
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

In [7]:
!ls -alh ./data/aclImdb/train/pos | wc -l

12503


In [8]:
train_texts, train_labels = read_imdb_split('./data/aclImdb/train')
test_texts, test_labels = read_imdb_split('./data/aclImdb/test')

print(len(train_texts))
print(len(train_labels))

25000
25000


In [9]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

print(len(train_texts))
print(type(train_texts))
print(train_texts[0])

20000
<class 'list'>
I grew up on Scooby Doo Where Are You, and I still love it. It is one of my favourite cartoons along with Darkwing Duck, Talespin, Peter Pan and the Pirates and Tom and Jerry. This show though is good for kids, the voices are good(Don Messick and Casey Kasem are perfect as Scooby and Shaggy), the theme tune is tolerable and it has some nice animation. However it is rather disappointing. I normally don't mind Scrappy, but when he appears to be like the main character, it gets annoying fast. Complete with the catchphrase Puppy Power, Scrappy is somewhat more annoying than usual. Also half the gang are missing after the first year, somehow it didn't feel like Scooby Doo. And the jokes and the story lines were in general lame and unoriginal, very little chasing monsters or unmasking the baddies. All in all, not as bad as Shaggy and Scooby Doo:Get a Clue, but this show is disappointing. 4/10 for the animation, voices, theme tune and the fact it is nice for kids. Bethany

In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [11]:
# see huggingface_predict
# batch encodings

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [12]:
# TODO: maybe for Trainer, TFTrainer only
# cause AutoModel use upper encodings directly

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

## Model - Normal

### mrpc

## Model - Trainer

### mrpc

### imdb

In [17]:
# TODO: 
# 1. param: warmup_steps
# 2. ./results file meaning
# 3. ./logs file meaning


training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [18]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [19]:
%time trainer.train()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


KeyboardInterrupt: 

## Save & Load

## Ref

[1] https://huggingface.co/transformers/training.html

dataset shuffle, batch, repeat <br>

[2] https://tensorflow.google.cn/api_docs/python/tf/data/Dataset#shuffle <br>
[3] https://tensorflow.google.cn/api_docs/python/tf/data/Dataset?hl=en#batch <br>
[4] https://tensorflow.google.cn/api_docs/python/tf/data/Dataset#repeat

dataset cardinality <br>

[5] https://tensorflow.google.cn/api_docs/python/tf/data/Dataset?hl=en#cardinality

[6] https://huggingface.co/transformers/custom_datasets.html

shell &> <br>

[7] https://stackoverflow.com/questions/13341702/how-do-i-turn-off-the-output-from-tar-commands-on-unix